# 要約 
このJupyter Notebookでは、LMSYS - Chatbot Arenaコンペティションにおける人間の好み予測の問題に取り組んでいます。具体的には、異なるバージョンのQwen2モデル（特に1.5bバージョン）を使用して、与えられたプロンプトに対する応答の好みを予測するためのトレーニングと推論を実行しています。ノートブックは、チャットボットの応答の選好を予測するモデルの性能向上を目的としています。

主な手法として、以下の手法とライブラリを使用しています：

1. **モデルとトークナイザー**: `transformers`ライブラリの`Qwen2ForSequenceClassification`と`AutoTokenizer`を利用し、Qwen2モデルを使用している。事前トレーニング済みのモデルを用いて、シーケンス分類タスクを実行します。

2. **トレーニング戦略**: `PeftModel`を使用して、パラメータ効率の良いファインチューニング（PEFT）を行う。特にLoRA（Low-Rank Adaptation）に基づくアプローチを使用し、モデルの特定の部分に対しての適応を行います。

3. **データの前処理**: ''sprompt''、''response_a''、''response_b''の各列に対してテキスト処理を行い、トークン化を実施しています。トークン化の際には、プロンプトや応答に対して特定の形式の前処理（接頭辞の追加）を行います。

4. **推論手法**: 並列処理を使用して、2つのモデルによって生成された応答に対する勝率を予測します。`ThreadPoolExecutor`を用いて、同時に複数のモデルでの推論を効率化しています。

5. **評価指標**: モデルの評価には対数損失および精度を使用し、これらの指標を用いてトレーニングと評価の過程でモデルの性能を評価します。

このノートブックは、最終的にテストデータに対する予測結果を保存したCSVファイル（"submission.csv"）を生成します。全体として、複数のモデルを活用して、与えられたプロンプトに対する応答の優劣を予測する精度を高めるためのフレームワークが整備されています。

---


# 用語概説 
以下は、ノートブック内で使われている専門用語の簡単な解説です。特に初心者がつまずきやすいマイナーな用語やドメイン特有の用語に焦点を当てています。

1. **SwiGLU活性化**:
   - SwiGLUは、Tanh活性化関数と線形変換を組み合わせた新しいタイプの活性化関数です。これにより、情報の流れを改善し、学習の効率を向上させることが期待されます。

2. **QKVバイアス**:
   - QKVは、Query、Key、Valueの略で、Transformerアーキテクチャにおける注意機構の基本要素です。QKVバイアスは、これらの要素にバイアス項を追加することで、モデルの性能を向上させる手法の一つです。

3. **グループクエリアテンション**:
   - グループクエリアテンションは、複数のクエリに対して同時に注意を払う技術です。これにより、計算効率が向上し、特定の情報に迅速にアクセスできるようになります。

4. **スライディングウィンドウアテンション**:
   - スライディングウィンドウアテンションは、入力シーケンスの局所部分に注目し、処理する手法です。これにより、長いシーケンスの処理が効率化され、計算リソースを節約できます。

5. **フルアテンション**:
   - フルアテンションは、入力シーケンス全体に対して注意を払う方法です。全てのトークン間の相互作用を考慮することで、モデルがより多くの情報を考慮できますが、その分計算コストが高くなります。

6. **LoRA (Low-Rank Adaptation)**:
   - LoRAは、大規模言語モデルを効率的に微調整するための手法で、低ランクの近似を利用してトレーニングパラメータの数を減らし、計算資源を節約することを目的としています。

7. **TTa (Test-Time Augmentation)**:
   - TTAは、推論時にデータを拡張し、その予測精度を向上させる手法です。異なる視点からのデータを使用することで、モデルのロバスト性を高める効果があります。

8. **注意マスク (Attention Mask)**:
   - 注意マスクは、Transformerモデルで、どの入力トークンに注意を払うべきかを示すためのバイナリマスクです。これにより、無関係なトークンを無視し、モデルが重要な情報に集中できるようにします。

9. **adaptive softmax**:
   - Adaptive softmaxは、多クラス分類問題において、クラス数が非常に多い場合に計算効率を向上させるための手法です。より頻繁に発生するクラスに対しては、高解像度の分布を学び、稀なクラスに対しては低解像度の分布を学びます。

10. **最大長 (max_length)**:
    - モデルが処理できる入力シーケンスの最大長を指定します。これにより、入力テキストが過剰な長さになることを防ぎ、計算リソースの消費を制御します。

これらの用語は、ノートブック内で特に重要な役割を果たすものであり、初心者が理解するための基礎になります。

---


# はじめに

このノートブックは[こちら](https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora)からフォークされたものです。

Qwen2は、異なるモデルサイズのデコーダー言語モデルを含む言語モデルシリーズです。各サイズごとに、基本の言語モデルと整合されたチャットモデルをリリースしています。これは、SwiGLU活性化、注意のQKVバイアス、グループクエリアテンション、スライディングウィンドウアテンションとフルアテンションの組み合わせなどに基づくトランスフォーマーアーキテクチャに基づいています。さらに、複数の自然言語やコードに適応した改善されたトークナイザーも搭載しています。

このノートブックでは、バッチサイズ4、1エポックでQwen2 1.5bバージョンのトレーニングを行っています。トレーニング時間はA100で約1時間です。Qwen2 7bはこのタスクに対してより良いパフォーマンスを発揮することが期待されます。トレーニング用のコードは、ノートブックの最後に添付されています。

In [ ]:
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files
# transformers, peft, accelerate, bitsandbytesのライブラリをインストールします。
# -Uオプションはアップグレードを意味し,
# --no-indexはPyPIからではなくローカルのリンクを使用することを指定します。
# --find-linksは指定したディレクトリ（この場合は/kaggle/input/lmsys-wheel-files）からパッケージを探します。

In [ ]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn

import numpy as np
import pandas as pd
from transformers import Qwen2ForSequenceClassification, AutoTokenizer, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel

# 必要なライブラリをインポートします。
# - time: 時間関連の関数を提供します。
# - dataclass: データクラスを作成するためのデコレーターです。
# - ThreadPoolExecutor: スレッドプールを使って非同期処理を実行するためのクラスです。
# - torch: PyTorchライブラリで、深層学習を行うための基本的なツールです。
# - sklearn: 機械学習用のライブラリです。
# - numpy: 数値計算ライブラリで、配列操作に使います。
# - pandas: データ分析のためのライブラリで、データフレームを扱います。
# - transformers: Hugging Faceのトランスフォーマーモデルに関連するライブラリです。
# - PeftModel: PEFT (Parameter-Efficient Fine-Tuning) に関連するモデルです。

In [ ]:
assert torch.cuda.device_count() == 2
# GPUの数が2であることを確認するためのアサーションです。
# torch.cuda.device_count()は使用可能なGPUの数を返します。
# この条件が満たされない場合、エラーが発生します。
# つまり、2つのGPUが必要な環境でこのコードを実行することを示しています。

## 設定

In [ ]:
@dataclass
class Config:
    model_dir = '/kaggle/input/qwen2/transformers/qwen2-1.5b-instruct/1'  # モデルの保存先ディレクトリ
    lora_dir = '/kaggle/input/lmsys-qwen2-1-5b-checkpoint/checkpoint-5748'  # LoRAチェックポイントの保存先ディレクトリ
    max_length = 2048  # 入力の最大長
    batch_size = 4  # バッチサイズ
    device = torch.device("cuda")  # 使用するデバイスをCUDA（GPU）に設定
    tta = False  # テスト時の拡張を有効にするかどうか。<prompt>-<model-bの応答>-<model-aの応答>
    spread_max_length = False  # 各入力にmax_length//3を適用するか、連結された入力にmax_lengthを適用するか

cfg = Config()  # 設定を初期化するためのConfigクラスのインスタンスを作成

# データのロードと前処理

In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  
# テストデータセットをCSVファイルから読み込みます。
# pandasのread_csv関数を使用して、指定されたパスのテストデータをDataFrameとして読み取ります。
# このDataFrameには、後のモデル推論で使用するデータが含まれています。

In [ ]:
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))  # 文字列を評価して、nullを空文字に置き換えた後、空白で結合します。

# データフレーム内の'sprompt'、'response_a'、'response_b'列のテキストを前処理します。
test.loc[:, 'prompt'] = test['prompt'].apply(process_text)  # 各プロンプトにprocess_text関数を適用して前処理を行う
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)  # 応答Aに対しても同様に前処理
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)  # 応答Bに対しても前処理

display(test.head(5))  # 前処理したデータの最初の5行を表示します。

# トークナイズ（トークン化）

In [ ]:
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    # プロンプトと応答をトークン化します。
    prompt = ["<User prompt>: " + p for p in prompt]  # 各プロンプトの先頭に"<User prompt>: "を追加
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]  # 応答Aの先頭に改行と"<response_a>: "を追加
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]  # 応答Bの先頭に改行と"<response_b>: "を追加
    
    if spread_max_length:
        # 各入力（プロンプト、応答A、応答B）に対してmax_lengthの1/3を適用
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        # トークンを結合し、入力IDを作成
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        attention_mask = [[1]* len(i) for i in input_ids]  # 各トークンに対する注意マスクを作成
    else:
        # プロンプト、応答A、応答Bを結合してトークン化
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)  # トークン化
        input_ids = tokenized.input_ids  # トークンのIDを取得
        attention_mask = tokenized.attention_mask  # 注意マスクを取得
    
    return input_ids, attention_mask  # トークンのIDと注意マスクを返す

In [ ]:
%%time

tokenizer = AutoTokenizer.from_pretrained(cfg.model_dir)  # 事前学習済みのトークナイザーをモデルディレクトリから読み込みます。
tokenizer.add_eos_token = True  # 終了トークンを追加します。
tokenizer.padding_side = "right"  # パディングを右側に設定します。

data = pd.DataFrame()  # 新しいデータフレームを作成します。
data["id"] = test["id"]  # テストデータのID列をコピーします。
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])  # トークン化を実行し、入力IDと注意マスクを取得します。
data["length"] = data["input_ids"].apply(len)  # 各入力IDの長さを計算して新しい列に保存します。

aug_data = pd.DataFrame()  # 拡張データ用の新しいデータフレームを作成します。
aug_data["id"] = test["id"]  # テストデータのID列をコピーします。
# 応答Aと応答Bを入れ替えます。
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])  # 応答を入れ替えてトークン化を実行します。
aug_data["length"] = aug_data["input_ids"].apply(len)  # 各入力IDの長さを計算して新しい列に保存します。

In [ ]:
print(tokenizer.decode(data["input_ids"][0]))  
# データフレームの最初の入力IDをデコードして、元のテキストを表示します。
# tokenizer.decode関数を使用して、トークンIDからそのテキスト表現を取得します。

In [ ]:
print(tokenizer.decode(aug_data["input_ids"][0]))  
# 拡張データフレームの最初の入力IDをデコードして、元のテキストを表示します。
# tokenizer.decode関数を使用して、トークンIDからそのテキスト表現を取得します。

# モデルのロード

In [ ]:
# GPU 0にベースモデルをロードします
device_0 = torch.device('cuda:0')  # GPU 0をデバイスとして指定します。
model_0 = Qwen2ForSequenceClassification.from_pretrained(
    cfg.model_dir,  # 設定されたモデルディレクトリから事前学習済みモデルを読み込みます。
    num_labels=3,  # 分類するラベルの数を指定します。
    device_map=device_0,  # モデルをGPU 0にマッピングします。
    use_cache=False,  # キャッシュを使用しない設定です。
)

# GPU 1にベースモデルをロードします
device_1 = torch.device('cuda:1')  # GPU 1をデバイスとして指定します。
model_1 = Qwen2ForSequenceClassification.from_pretrained(
    cfg.model_dir,  # 設定されたモデルディレクトリから事前学習済みモデルを読み込みます。
    num_labels=3,  # 分類するラベルの数を指定します。
    device_map=device_1,  # モデルをGPU 1にマッピングします。
    use_cache=False,  # キャッシュを使用しない設定です。
)

#### LoRAアダプターのロード

In [ ]:
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)  # LoRAアダプターをモデル0にロードします。
model_0.config.pad_token_id = model_0.config.eos_token_id  # パディングトークンIDを終了トークンIDに設定します。

model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)  # LoRAアダプターをモデル1にロードします。
model_1.config.pad_token_id = model_1.config.eos_token_id  # パディングトークンIDを終了トークンIDに設定します。

# 推論

In [ ]:
@torch.no_grad()  # 勾配計算を無効にします。推論時に計算を節約できます。
@torch.cuda.amp.autocast()  # 自動混合精度を使用して、計算の効率を向上させます。
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # モデルAの勝ち、モデルBの勝ち、引き分けの結果を格納するリストを初期化します。
    
    for start_idx in range(0, len(df), batch_size):  # データフレームをバッチサイズごとに処理します。
        end_idx = min(start_idx + batch_size, len(df))  # 現在のバッチの終了インデックスを計算します。
        tmp = df.iloc[start_idx:end_idx]  # 現在のバッチを取得します。
        input_ids = tmp["input_ids"].to_list()  # 入力IDをリストに変換します。
        attention_mask = tmp["attention_mask"].to_list()  # 注意マスクをリストに変換します。
        
        # トークンのパディングを行います。注意警告を表示しないようにします。
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",  # 最も長い入力に合わせてパディングします。
            pad_to_multiple_of=None,
            return_tensors="pt",  # PyTorchのテンソルとして返します。
        )
        
        # モデルによる出力を取得します。
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()  # ロジットからソフトマックスを計算し、CPUに移動します。
        
        # 各モデルの勝率をリストに追加します。
        a_win.extend(proba[:, 0].tolist())  # モデルAの勝率
        b_win.extend(proba[:, 1].tolist())  # モデルBの勝率
        tie.extend(proba[:, 2].tolist())  # 引き分けの確率
    
    # データフレームに結果を追加します。
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df  # 更新されたデータフレームを返します。

In [ ]:
st = time.time()  # 処理開始時間を記録します。

# 入力の長さに基づいてソートし、動的パディングを最大限に活用します。
data = data.sort_values("length", ascending=False)  
# total #tokens in sub_1 and sub_2はほぼ同じである必要があります。
sub_1 = data.iloc[0::2].copy()  # データを偶数行で取得してサブセット1を作成します。
sub_2 = data.iloc[1::2].copy()  # データを奇数行で取得してサブセット2を作成します。

# ThreadPoolExecutorを使用して、モデル0とモデル1の推論を並列で実行します。
with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 推論を実行

result_df = pd.concat(list(results), axis=0)  # 結果をデータフレームに結合します。
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 勝率のデータを抽出します。

print(f"elapsed time: {time.time() - st}")  # 経過時間を表示します。

In [ ]:
st = time.time()  # 処理開始時間を記録します。

if cfg.tta:  # テスト時の拡張が有効な場合
    data = aug_data.sort_values("length", ascending=False)  # 入力の長さに基づいてソートして速度を向上させます。
    sub_1 = data.iloc[0::2].copy()  # データを偶数行で取得してサブセット1を作成します。
    sub_2 = data.iloc[1::2].copy()  # データを奇数行で取得してサブセット2を作成します。

    # ThreadPoolExecutorを使用して、モデル0とモデル1の推論を並列で実行します。
    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 推論を実行

    tta_result_df = pd.concat(list(results), axis=0)  # 結果をデータフレームに結合します。
    # TTAの順序は反転しています。
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values  # TTAの結果を取得します。
    # 元の結果とTTA結果の平均を取ります。
    proba = (proba + tta_proba) / 2  # 勝率の平均を計算します。

print(f"elapsed time: {time.time() - st}")  # 経過時間を表示します。

In [ ]:
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # 勝率のデータを結果データフレームに更新します。
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝率を更新します。
result_df.loc[:, "winner_tie"] = proba[:, 2]  # 引き分けの勝率を更新します。
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]  # 提出用データフレームを作成します。
submission_df.to_csv('submission.csv', index=False)  # 提出ファイルをCSV形式で保存します。
display(submission_df)  # 提出データフレームを表示します。

# トレーニングコード

このトレーニングコードは、https://www.kaggle.com/code/emiz6413/training-gemma-2-9b-4-bit-qlora-fine-tuning からフォークされ、インスパイアを受けています。

## Kaggleデータの転送

In [ ]:
# # 重要: このセルを実行してKaggleデータソースを正しい位置（/kaggle/input）にインポートします。
# # その後、このセルは削除しても問題ありません。
# # 注意: このノートブック環境はKaggleのPython環境とは異なるため、
# # お使いのノートブックで使用されるライブラリが欠落している可能性があります。

# import os
# import sys
# from tempfile import NamedTemporaryFile
# from urllib.request import urlopen
# from urllib.parse import unquote, urlparse
# from urllib.error import HTTPError
# from zipfile import ZipFile
# import tarfile
# import shutil

# CHUNK_SIZE = 40960
# DATA_SOURCE_MAPPING = 'lmsys-chatbot-arena:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F66631%2F8346466%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T010448Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4ae2cd10c5a4750deca45551519904c5858980d5cf8cd8ade09b2299d926c86c895b50ba333acf0db5210d0dd29197c9a9c5a525c8afd0186b88f17d3ca756f0562ad5acfa2e856e8b159f554e61f72102865abd60add751dd59bed5126536d977d6fe54d2e85f8e5baa8d3d75337d0a222a89f0f30fa6dd7c360e4a192363dc417e9e4a9c9c23368991db65b4994c2200bee494d8d5e2684f754ab1b1a511f7db3652e01ab658b04d26cc1321e783fa5509f67d4c438808adc7932a0e79a21849375023b36e90cbe288cf68a6185b2ce950464b71c9b6133d49769c67e77a5298809fb63da23c0655165e80661623bd9bb908bc9a486dbc9e09caebf2a01392,qwen2/transformers/qwen2-1.5b-instruct/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F52038%2F62308%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240716%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240716T010448Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D80f6a1f964073d960129f611693afca0666128c1a319b7ab93769a1993d5910e4995cf3c6f2f87323452999b069eafb9f7b01be97dbb80a3441cd0b4871d9d35379750a3b5614397253624b097961c886c48df889ac7b1100231715e2b60bf4f3ffccd5e7fc68b3d7d0668f350a8fefbddb90275770e75aaa7f74fae68b2f5314f610ec2f1abf0436156e9426e6173e229172ca0c4ee91eb2d768de3190c9f07e6c28b73bc8c5553e2dac6320842103524591b663021a41801bb2274b5fd91dd62f174ba8976c74995012ad3ed34ecf554a9e2cb08f91813e9cacc9b8d554c6a7a037414635f30e506ea39f63fb4db01f5cf3322dca02097f9550b1a5454ae99'

# KAGGLE_INPUT_PATH='/kaggle/input'
# KAGGLE_WORKING_PATH='/kaggle/working'
# KAGGLE_SYMLINK='kaggle'

# !umount /kaggle/input/ 2> /dev/null
# shutil.rmtree('/kaggle/input', ignore_errors=True)
# os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
# os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

# try:
#   os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
# except FileExistsError:
#   pass
# try:
#   os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
# except FileExistsError:
#   pass

# for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
#     directory, download_url_encoded = data_source_mapping.split(':')
#     download_url = unquote(download_url_encoded)
#     filename = urlparse(download_url).path
#     destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
#     try:
#         with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
#             total_length = fileres.headers['content-length']
#             print(f'Downloading {directory}, {total_length} bytes compressed')
#             dl = 0
#             data = fileres.read(CHUNK_SIZE)
#             while len(data) > 0:
#                 dl += len(data)
#                 tfile.write(data)
#                 done = int(50 * dl / int(total_length))
#                 sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
#                 sys.stdout.flush()
#                 data = fileres.read(CHUNK_SIZE)
#             if filename.endswith('.zip'):
#               with ZipFile(tfile) as zfile:
#                 zfile.extractall(destination_path)
#             else:
#               with tarfile.open(tfile.name) as tarfile:
#                 tarfile.extractall(destination_path)
#             print(f'\nDownloaded and uncompressed: {directory}')
#     except HTTPError as e:
#         print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
#         continue
#     except OSError as e:
#         print(f'Failed to load {download_url} to path {destination_path}')
#         continue

# print('データソースのインポートが完了しました。')

## ライブラリのインストールとロード

In [ ]:
## gemma-2はtransformers>=4.42.3から利用可能です。
# !pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft datasets
# transformersライブラリのバージョンを4.42.3以上にアップグレードし、その他の必要なライブラリ（bitsandbytes、accelerate、peft、datasets）をインストールします。

In [ ]:
# import os  # オペレーティングシステムの機能を提供するライブラリをインポート
# import copy  # オブジェクトのコピーを作成するためのライブラリをインポート
# from dataclasses import dataclass  # データクラスの作成に使用するデコレーターをインポート

# import numpy as np  # 数値計算ライブラリをインポート
# import torch  # 深層学習ライブラリPyTorchをインポート
# from datasets import Dataset  # データセットの扱いに使用するクラスをインポート
# from transformers import (  # Transformersライブラリから必要なクラスをインポート
#     BitsAndBytesConfig,
#     AutoTokenizer,
#     Qwen2ForSequenceClassification,
#     PreTrainedTokenizerBase,
#     EvalPrediction,
#     Trainer,
#     TrainingArguments,
#     DataCollatorWithPadding,
# )
# from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType  # PEFTに関連するクラスをインポート
# from sklearn.metrics import log_loss, accuracy_score  # 機械学習の評価指標をインポート

## トレーニング設定

In [ ]:
# @dataclass
# class Config:
#     output_dir: str = "output"  # 出力先ディレクトリの指定
#     model_name: str = "/kaggle/input/qwen2/transformers/qwen2-1.5b-instruct/1"  # 使用するモデルのパス
#     checkpoint: str = "/kaggle/input/qwen2/transformers/qwen2-1.5b-instruct/1"  # Qwenモデルのローカルディレクトリ
#     max_length: int = 2048  # 入力の最大長
#     n_splits: int = 5  # クロスバリデーションの分割数
#     fold_idx: int = 0  # 現在のフォールドインデックス
#     optim_type: str = "adamw_8bit"  # オプティマイザーの種類
#     per_device_train_batch_size: int = 4  # デバイスごとのトレーニングバッチサイズ
#     gradient_accumulation_steps: int = 2  # グローバルバッチサイズは8
#     per_device_eval_batch_size: int = 8  # デバイスごとの評価バッチサイズ
#     n_epochs: int = 1  # エポック数
#     freeze_layers: int = 16  # 最初の16層にはアダプターを追加しない
#     lr: float = 2e-4  # 学習率
#     warmup_steps: int = 20  # ウォームアップステップ数
#     lora_r: int = 16  # LoRAのR値
#     lora_alpha: float = lora_r * 2  # LoRAのアルファ値
#     lora_dropout: float = 0.05  # LoRAのドロップアウト率
#     lora_bias: str = "none"  # LoRAのバイアス設定

# config = Config()  # 設定のインスタンスを作成

## トレーニング引数

In [ ]:
# training_args = TrainingArguments(
#     output_dir="output",  # 出力先ディレクトリの指定
#     overwrite_output_dir=True,  # 出力ディレクトリを上書きするかどうか
#     report_to="none",  # ログの報告先（ここでは無効化）
#     num_train_epochs=config.n_epochs,  # トレーニングエポック数
#     per_device_train_batch_size=config.per_device_train_batch_size,  # デバイスごとのトレーニングバッチサイズ
#     gradient_accumulation_steps=config.gradient_accumulation_steps,  # 勾配累積のステップ数
#     per_device_eval_batch_size=config.per_device_eval_batch_size,  # デバイスごとの評価バッチサイズ
#     logging_steps=10,  # ロギングの頻度
#     eval_strategy="epoch",  # 評価戦略（エポックごとに評価）
#     save_strategy="steps",  # モデル保存戦略（ステップごとに保存）
#     save_steps=200,  # モデルを保存するステップ数
#     optim=config.optim_type,  # 使用するオプティマイザー
#     fp16=True,  # 半精度学習を有効にするかどうか
#     learning_rate=config.lr,  # 学習率
#     warmup_steps=config.warmup_steps,  # ウォームアップステップ数
# )

## LoRA設定

In [ ]:
# lora_config = LoraConfig(
#     r=config.lora_r,  # LoRAのR値
#     lora_alpha=config.lora_alpha,  # LoRAのアルファ値
#     # 自己注意のみを対象とする
#     target_modules=["q_proj", "k_proj", "v_proj"],  # ターゲットモジュール（自己注意のプロジェクション）
#     layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],  # 変換するレイヤーのリスト（フリーズされていないレイヤー）
#     lora_dropout=config.lora_dropout,  # LoRAのドロップアウト率
#     bias=config.lora_bias,  # LoRAのバイアス設定
#     task_type=TaskType.SEQ_CLS,  # タスクのタイプ（シーケンス分類）
# )

## モデルの初期化とトークン化

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(config.model_name)  # 設定されたモデル名からトークナイザーを初期化します。
# tokenizer.add_eos_token = True  # 終了トークンを追加します。
# tokenizer.padding_side = "right"  # パディングを右側に設定します。
# model = Qwen2ForSequenceClassification.from_pretrained(
#     config.model_name,  # 設定されたモデル名からモデルを初期化します。
#     num_labels=3,  # 分類するラベルの数
#     torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
#     device_map="auto",  # 自動的にデバイスをマッピングします。
# )
# model.config.use_cache = False  # キャッシュの使用を無効にします。
# model = prepare_model_for_kbit_training(model)  # モデルを8bitトレーニング用に準備します。
# model = get_peft_model(model, lora_config)  # LoRAモデルを取得します。
# model.config.pad_token_id = model.config.eos_token_id  # パディングトークンIDを終了トークンIDに設定します。
# model  # 初期化されたモデルを表示します。

In [ ]:
# model.print_trainable_parameters()  # トレーニング可能なパラメータを表示します。
# この関数は、モデル内で訓練されるパラメータの数や詳細を示します。

# トレーニングデータのロード

In [ ]:
# ds = Dataset.from_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")  # トレーニングデータをCSVファイルから読み込みます。
# class CustomTokenizer:
#     def __init__(
#         self,
#         tokenizer: PreTrainedTokenizerBase,  # トークナイザーの初期化
#         max_length: int  # 最大長の設定
#     ) -> None:
#         self.tokenizer = tokenizer  # トークナイザーを保存
#         self.max_length = max_length  # 最大長を保存

#     def __call__(self, batch: dict) -> dict:  # バッチデータを処理するメソッド
#         prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]  # プロンプトに接頭辞を追加
#         response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]  # 応答Aに接頭辞を追加
#         response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]  # 応答Bに接頭辞を追加
#         texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # テキストを結合
#         tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)  # トークン化
#         labels = []  # ラベルを初期化
#         for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):  # 勝率に基づくラベルを設定
#             if a_win:
#                 label = 0  # モデルAが勝つ場合
#             elif b_win:
#                 label = 1  # モデルBが勝つ場合
#             else:
#                 label = 2  # 引き分けの場合
#             labels.append(label)  # ラベルを追加
#         return {**tokenized, "labels": labels}  # トークン化されたデータとラベルを返す

#     @staticmethod
#     def process_text(text: str) -> str:  # テキストを処理する静的メソッド
#         return " ".join(eval(text, {"null": ""}))  # nullを空文字に置き換えた後、空白で結合します。
# encode = CustomTokenizer(tokenizer, max_length=config.max_length)  # カスタムトークナイザーのインスタンス化
# ds = ds.map(encode, batched=True)  # データセットにトークナイザーを適用します。

## トレーニングの実行

In [ ]:
# def compute_metrics(eval_preds: EvalPrediction) -> dict:  # 評価指標を計算する関数
#     preds = eval_preds.predictions  # 予測結果を取得
#     labels = eval_preds.label_ids  # ラベルを取得
#     probs = torch.from_numpy(preds).float().softmax(-1).numpy()  # ソフトマックスを適用して確率を計算
#     loss = log_loss(y_true=labels, y_pred=probs)  # ログ損失を計算
#     acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))  # 精度を計算
#     return {"acc": acc, "log_loss": loss}  # 精度とログ損失を辞書として返す

# folds = [  # クロスバリデーションのためのフォールドを作成
#     (
#         [i for i in range(len(ds)) if i % config.n_splits != fold_idx],  # トレーニングインデックス
#         [i for i in range(len(ds)) if i % config.n_splits == fold_idx]  # 検証インデックス
#     )
#     for fold_idx in range(config.n_splits)  # 注目するフォールドのインデックス
# ]

In [ ]:
# train_idx, eval_idx = folds[config.fold_idx]  # 指定されたフォールドインデックスに基づいてトレーニングと評価のインデックスを取得

# trainer = Trainer(  # トレーニング用のTrainerインスタンスを作成
#     args=training_args,  # トレーニング引数
#     model=model,  # トレーニングするモデル
#     tokenizer=tokenizer,  # 使用するトークナイザー
#     train_dataset=ds.select(train_idx),  # トレーニングデータセット
#     eval_dataset=ds.select(eval_idx),  # 評価データセット
#     compute_metrics=compute_metrics,  # 評価指標の計算を行う関数
#     data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # パディングを行うデータコレータ
# )
# trainer.train()  # トレーニングを開始

---

# コメント

> ## Jiadi Wang トピック著者
> 
> これはトレーニングの損失データです。
> 
> もしよろしければ、UPVOTEしていただけると非常に助かります！ありがとうございます！
> 
> 

---